In [10]:
#pip install matplotlib

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
root_path = '/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/'
#read in the long format csv
EEO1_data = pd.read_csv(root_path + 'amz_long_csv.csv')

In [14]:
EEO1_data = pd.DataFrame(EEO1_data)
EEO1_data = EEO1_data.drop(columns = ['Unnamed: 0'])

EEO1_data

,count_employees,job_category,gender,race,year,company
0,106,Exec/Sr. Officials & Mgrs,Male,Hispanic or Latino,2021,Amazon
1,5192,First/Mid Officials & Mgrs,Male,Hispanic or Latino,2021,Amazon
2,4785,Professionals,Male,Hispanic or Latino,2021,Amazon
3,1240,Technicians,Male,Hispanic or Latino,2021,Amazon
4,9819,Sales Workers,Male,Hispanic or Latino,2021,Amazon
...,...,...,...,...,...,...
415,699,Administrative Support,Female,Two or More Races,2020,Amazon
416,2,Craft Workers,Female,Two or More Races,2020,Amazon
417,652,Operatives,Female,Two or More Races,2020,Amazon
418,11157,Laborers & Helpers,Female,Two or More Races,2020,Amazon
